In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc('figure', figsize=(16,12))
mpl.rc('font', size=20.)
mpl.rc('font', family='serif')
mpl.rc('text', usetex=True)

In [ ]:
yr = 3600 * 24 * 365.25
gs = 500
s = 1
cool = plt.cm.Blues_r
warm = plt.cm.autumn

In [ ]:
simname = {'vanilla':r'$u_{\rm \tiny CR} = 0$',
        'cr00': r'$u_{\rm \tiny CR} = u_0$', 
        'cr01': r'$u_{\rm \tiny CR} = 10 u_0$', 
        'cr02': r'$u_{\rm \tiny CR} = 10^2 u_0$', 
        'cr03': r'$u_{\rm \tiny CR} = 10^3 u_0$', 
        'cr04': r'$u_{\rm \tiny CR} = 10^4 u_0$',
        'cr05': r'$u_{\rm \tiny CR} = 10^5 u_0$'}

In [ ]:
sims = ['vanilla', 'cr00', 'cr01', 'cr02', 'cr03', 'cr04', 'cr05']
snaps = [355, 310, 327, 318, 249, 213, 176]
cooltime = {}
heattime = {}

In [ ]:
for sim, snap in zip(sims, snaps):
    cooltime[sim] = pd.read_csv(os.getenv('HOME')+'/sim/chemistry/stampede/'+sim+'/cooltime_{:0>4}.dat'.format(snap),
                                delim_whitespace=True, names=['a', 'id', 'n', 'temp', 'tchem'])
    heattime[sim] = pd.read_csv(os.getenv('HOME')+'/sim/chemistry/stampede/'+sim+'/heattime_{:0>4}.dat'.format(snap),
                                delim_whitespace=True, names=['a', 'id', 'n', 'temp', 'tchem'])

In [ ]:
f,axes = plt.subplots(3,2, sharex=True, sharey=True, figsize=(16,16))
ndens = np.logspace(-3, 12, 100)
rho = 1.6726e-24 * ndens
tff = np.sqrt(3 / (32 * np.pi * 6.67e-8 * rho))

for sim,ax in zip(sims[1:],f.axes):
    ax.hexbin(cooltime['vanilla'].n.values[::s],cooltime['vanilla'].tchem.values[::s]/yr,
              gridsize=gs, bins='log', xscale='log', yscale='log', mincnt=1, color='.75')
    ax.hexbin(heattime['vanilla'].n.values[::s],heattime['vanilla'].tchem.values[::s]/yr,
              gridsize=gs, bins='log', xscale='log', yscale='log', mincnt=1, color='.75')

    ax.hexbin(cooltime[sim].n.values[::s],cooltime[sim].tchem.values[::s]/yr,
              gridsize=gs, bins='log', xscale='log', yscale='log', mincnt=1, cmap=cool)
    ax.hexbin(heattime[sim].n.values[::s],heattime[sim].tchem.values[::s]/yr,
              gridsize=gs, bins='log', xscale='log', yscale='log', mincnt=1, cmap=warm)

    ax.plot(ndens,tff/yr, 'k', lw=2.5, label='freefall time')
    ax.text(.97,.95, simname[sim], ha='right', va='top', size=25, transform=ax.transAxes)
ax.set_ylim(20,9e9)
ax.set_xlim(2e-2,9e11)
for ax in axes[:,0]:
    ax.set_ylabel('Heating Time [yr]')
for ax in axes[-1,:]:
    ax.set_xlabel('Number Density [cm$^{-3}$]')

f.tight_layout(w_pad=0.0,h_pad=0.0)
f.savefig('figures/phase_diagrams/cooltime.png', bbox_inches='tight')